In [1]:
import ROOT
from array import array
import matplotlib.pyplot as plt
import numpy as np

Welcome to JupyROOT 6.28/06


In [2]:
def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return "{:d} ({:.1f}%)".format(absolute, pct)

In [3]:
infile = ROOT.TFile.Open("InBodyExcelData_2022-09-08_18-03-19_ridotto_ML_new.root")
tree = infile.Get("t1")

Error in <TNetXNGFile::Open>: [ERROR] Server responded with an error: [3010] Unable to give access - user access restricted - unauthorized identity used ; Permission denied


In [4]:
def first_last_array(tree, clust, branchname):
    pre_id = -9999
    last_value = -9999
    first_values = []
    last_values = []
    for entry in tree:
        if entry.Local_ID == 42:
            continue
        id = entry.Local_ID
        if pre_id != id and last_value != -9999:
            if(cluster == clust or clust < 0):
                first_values.append(first_value)
                last_values.append(last_value)
        if(pre_id == id):
            last_value = getattr(entry, branchname)
            nr_visite = entry.nr_visite
        else:
            last_value = -9999
        if pre_id != id:
            cluster = entry.cluster
            pre_id = id
            first_value = getattr(entry, branchname)
    return first_values, last_values


In [5]:
def first_last_array2var(tree, clust, branchname1, branchname2, optype):
    pre_id = -9999
    first_values = []
    last_values = []
    for entry in tree:
        if entry.Local_ID == 42:
            continue
        id = entry.Local_ID
        if pre_id == -9999:
            cluster = entry.cluster
            pre_id = id
            first_value1 = getattr(entry, branchname1)
            first_value2 = getattr(entry, branchname2)
        if pre_id != id and last_value1 != -9999:
            if(cluster == clust or clust < 0):
                if(optype == 'sub'):
                    first_values.append(first_value1-first_value2)
                    last_values.append(last_value1-last_value2)
                elif(optype == 'ratio' or optype == 'r'):
                    first_values.append(first_value1/first_value2*100)
                    last_values.append(last_value1/last_value2*100)
        if(pre_id == id):
            last_value1 = getattr(entry, branchname1)
            last_value2 = getattr(entry, branchname2)
            nr_visite = entry.nr_visite
        else:
            last_value1 = -9999
        if pre_id != id:
            cluster = entry.cluster
            pre_id = id
            first_value1 = getattr(entry, branchname1)
            first_value2 = getattr(entry, branchname2)
    return first_values, last_values


In [6]:
def var2lab(var):
    if 'FFM_' in var or 'BFM_' in var:
        lab = var.replace('FFM_right_arm', 'FFM braccio destro').replace('FFM_left_arm', 'FFM braccio sinistro').replace('FFM_trunk', 'FFM tronco').replace('FFM_right_leg', 'FFM gamba destra').replace('FFM_left_leg', 'FFM gamba sinistra').replace('BFM_right_arm', 'BFM braccio destro').replace('BFM_left_arm', 'BFM braccio sinistro').replace('BFM_trunk', 'BFM tronco').replace('BFM_right_leg', 'BFM gamba destra').replace('BFM_left_leg', 'BFM gamba sinistra')
    else: 
        lab = var.replace('Gender', 'Sesso').replace('Height', 'Altezza [cm]').replace('Age', 'Età').replace('Weight', 'Peso [kg]').replace('Protein', 'Proteine').replace('Minerals', 'Minerali').replace('SLM', 'SLM').replace('BMI', 'BMI [kg/m$^2$]').replace('PBF', 'PBF').replace('WHR', 'WHR').replace('cluster', 'Cluster').replace('BMR', 'BMR').replace('Target_weight', 'Peso target [kg]').replace('Obesity_degree', 'Grado di obesità').replace('Recom_cal_intake', 'Calorie raccomandate').replace('ICW', 'ICW [\%]').replace('ECW', 'ECW [\%]').replace('Neck_circ', 'Circonferenza collo [cm]').replace('Chest_circ', 'Circonferenza torace [cm]').replace('Abdomen_circ', 'Circonferenza addome [cm]').replace('Hip_circ', 'Circonferenza fianchi [cm]').replace('RightArm_circ', 'Circonferenza braccio destro [cm]').replace('LeftArm_circ', 'Circonferenza braccio sinsitro [cm]').replace('RightThigh_circ', 'Circonferenza coscia destra [cm]').replace('LeftThigh_circ', 'Circonferenza coscia sinistra [cm]').replace('VFL', 'VFL').replace('AMC', 'AMC').replace('BMC', 'BMC').replace('SMI', 'SMI').replace('BFM', 'BFM [\%]').replace('FFM', 'FFM [\%]').replace('SMM', 'SMM').replace('BCM', 'BCM').replace('TBW', 'TBW').replace('Arm_circ', 'Circonferenza braccio [cm]')
    return lab


In [7]:
def clust2tag(clust):
    tag = clust.replace("10_",  "10 - Misto, Sottopeso").replace("1_",  "1 - U, Obesità grave").replace("2_",  "2 - D, Sovrappeso, 44 anni").replace("3_",  "3 - D, Normopeso, 55 anni").replace("4_",  "4 - Misto, Obesità grave").replace("5_",  "5 - U, Leggero Sovrappeso").replace("6_",  "6 - D, Sovrappeso, 27 anni").replace("7_",  "7 - D, Normopeso, 35 anni").replace("8_",  "8 - U, Molto Sovrappeso").replace("9_",  "9 - U, Sovrappeso")
    return tag

In [8]:
leaves = [leaf.GetName() for leaf in tree.GetListOfLeaves()]
#leaves.append()
leaves.remove('Year')
leaves.remove('Month')
leaves.remove('Day')
leaves.remove('Local_ID')
leaves.remove('nr_visite')
leaves.remove('v_BMI_score')
leaves.remove('v_WHR_score')
leaves.remove('v_FFM_score')
leaves.remove('v_TOT_score')
leaves.remove('grid')
firstvis_val = {}
lastvis_val = {}
circ = ['Arm_circ', 'Neck_circ', 'Chest_circ', 'Abdomen_circ', 'Hip_circ', 'RightThigh_circ'] #'RightArm_circ', 'LeftArm_circ',  'LeftThigh_circ'
leaf_weight = ['BFM', 'FFM', 'SMM', 'BCM', 'TBW'] # 'SLM',
leaf_ICW = ['ECW']
[leaves.remove(leaf) for leaf in circ]
[leaves.remove(leaf) for leaf in leaf_weight]
[leaves.remove(leaf) for leaf in leaf_ICW]


[None]

In [9]:
[print(var2lab(leaf)) for leaf in circ]


Circonferenza braccio [cm]
Circonferenza collo [cm]
Circonferenza torace [cm]
Circonferenza addome [cm]
Circonferenza fianchi [cm]
Circonferenza coscia destra [cm]


[None, None, None, None, None, None]

In [10]:
print(leaves)

['Gender', 'DeltaDays', 'Height', 'Age', 'Weight', 'ICW', 'Protein', 'Minerals', 'SLM', 'BMI', 'PBF', 'FFM_right_arm', 'FFM_left_arm', 'FFM_trunk', 'FFM_right_leg', 'FFM_left_leg', 'BFM_right_arm', 'BFM_left_arm', 'BFM_trunk', 'BFM_right_leg', 'BFM_left_leg', 'Target_weight', 'BMR', 'WHR', 'VFL', 'Obesity_degree', 'AMC', 'BMC', 'RightArm_circ', 'LeftArm_circ', 'LeftThigh_circ', 'SMI', 'Recom_cal_intake', 'cluster']


In [23]:
for i,leaf in enumerate(circ):
    firstvis_val[leaf] = []
    lastvis_val[leaf] = []
    for cluster in range(1, 11):
        firstvis, lastvis = first_last_array(tree, cluster, leaf)
        firstvis_val[leaf].append(firstvis)
        lastvis_val[leaf].append(lastvis)


In [24]:
for i,leaf in enumerate(leaves):
    firstvis_val[leaf] = []
    lastvis_val[leaf] = []
    for cluster in range(1, 11):
        firstvis, lastvis = first_last_array(tree, cluster, leaf)
        firstvis_val[leaf].append(firstvis)
        lastvis_val[leaf].append(lastvis)

In [25]:
for i,leaf in enumerate(leaf_weight):
    firstvis_val[leaf] = []
    lastvis_val[leaf] = []
    for cluster in range(1, 11):
        firstvis, lastvis = first_last_array2var(tree, cluster, leaf, 'Weight', 'r')
        firstvis_val[leaf].append(firstvis)
        lastvis_val[leaf].append(lastvis)
    print(firstvis_val[leaf])

[[50.933786957767865, 52.6235361415257, 48.85411096014377, 48.79097263503126], [50.847457627118644, 44.82029831900886, 49.95102915004135, 44.25228753766771, 48.65168453602309, 47.14867685939305, 47.483589498180336, 53.22740824080704, 48.46885217004318, 49.05008440429254, 51.2422368342593, 51.72745019623359, 45.89632905002419, 50.28625880565326, 49.61154441726032, 47.634407781785534, 51.48304839888995, 53.33333249951973, 45.33493088233858, 47.87986062201134, 43.95721761938085, 49.08722340252585, 45.29540572727119, 45.057231971466535, 51.02739902004083, 48.728382542021045, 50.381681898000394, 51.658292990833075, 50.61452428721849, 49.42119188938995, 50.840339367019894, 51.15848005775996, 51.56118014693764, 51.83603890170336, 51.090343919336625, 46.48351564512148, 51.31048698765191, 52.90806973094932, 46.44549763033176, 42.81376308753865, 53.567935095109384, 48.40579562717014, 45.41522551295964, 53.75347468006216, 48.42319124757128, 51.21268944847589, 50.70993993283193, 51.727115671237144

[[49.066213042232135, 47.3764638584743, 51.14589330447043, 51.20902736496874], [49.152542372881356, 55.17970571344074, 50.04897084995865, 55.74770858166378, 51.34831546397691, 52.851327025227505, 52.516410501819664, 46.77259175919296, 51.53115185814836, 50.94991230149575, 48.757763165740705, 48.27255346470438, 54.1036709499758, 49.713737554368535, 50.388459816587996, 52.36559221821446, 48.516947560117245, 46.66666750048027, 54.66507368069651, 52.12014274786265, 56.04278238061915, 50.912780466335526, 54.70459427272881, 54.942768028533465, 48.97260533463649, 51.271617457978955, 49.618322261974946, 48.34170700916693, 49.38547571278151, 50.578804713729234, 49.15966464001518, 48.84151994224004, 48.43881985306237, 48.16396435593738, 48.90965905161155, 53.51648435487852, 48.68951685780915, 47.09193384756598, 53.55450236966824, 57.18623305143184, 46.432061175958886, 51.594204372829864, 54.584774487040356, 46.246525319937845, 51.57680487176021, 48.78731411001046, 49.29006006716808, 48.272884328

[[27.95699059369471, 27.050431812841282, 29.23420962625774, 29.124126268320182], [26.779661879027632, 30.76110026516582, 27.815866839340565, 30.72227856090531, 27.752809845999387, 29.633402531374138, 28.665208233635763, 26.017876421988113, 28.51108856398033, 28.497408575412877, 27.22567251810698, 26.58349479290147, 30.129589717355643, 27.576334710846684, 27.524971872428733, 29.13978535641906, 27.33050722460816, 25.792350143682764, 30.14354213264761, 29.328623372772096, 31.016042780748666, 28.397566362385877, 30.634572792728136, 30.176899542625335, 26.712328796949848, 28.280771489318308, 27.044711082224243, 26.83417162104468, 26.81564245810056, 27.871770602387897, 26.78571514436466, 26.87673734003108, 27.004219409282697, 26.64389510631343, 27.49221255066358, 30.109889690692608, 26.713710499231603, 25.984991706790773, 29.57346043880517, 32.48987599454328, 25.708698389534053, 28.695652911052612, 30.709342965906046, 25.393882512558275, 28.789418472509887, 26.772389533552186, 27.58620809723

[[31.975099590154272, 30.820172725136512, 33.37059966446015, 33.15421748488265], [31.864407641739493, 36.152221262768045, 32.71302842805103, 36.01220796223147, 32.921347457371404, 34.7250504417979, 33.916848449091866, 30.78450937385171, 33.57972572527186, 33.160622205437846, 32.09109781539471, 31.285987935598534, 35.4211660664629, 32.44274714687589, 32.630410784051975, 34.40860215053764, 32.309321511651355, 30.71038293056801, 35.64593275236162, 34.18728074757284, 36.47058660333807, 33.36714202334796, 35.995624700907285, 35.37981325687108, 31.849315187799377, 33.36724132123413, 32.06106935327859, 31.65829145728643, 31.95530768879299, 32.59127113046699, 31.82773131130864, 31.510657571070922, 31.476792605114383, 31.169940628194322, 31.93146569232243, 35.49450465611049, 31.552419556415295, 30.58161251478374, 34.59715639810427, 37.955464414667155, 30.40078147967889, 33.623187668657536, 35.6401395387239, 29.93512398543726, 33.875888304425715, 31.43656877080914, 32.55577988933354, 30.74265763

[[36.502547319733644, 35.20122129386912, 37.7864733261176, 37.82912370605719], [36.27118471652101, 40.803381709018275, 36.92458504051313, 40.895218224736965, 37.9775272326523, 38.900205651812136, 38.84026193363746, 34.657400778092196, 37.909189738939354, 37.478411383530705, 35.81780436921058, 35.60460610430657, 39.95680411413871, 36.736640151609464, 37.18091072956234, 38.60215217836441, 35.69915277353137, 34.53551954258987, 40.31100643305813, 38.33922499637738, 41.60427970682236, 37.829614416120336, 40.3719922427256, 40.58272697111683, 36.18721611314218, 37.741604600548044, 36.968378030604434, 35.577890980782826, 36.64804384029111, 37.3107750554269, 35.92437170090785, 36.05190081165019, 35.86497890295359, 35.69598615007521, 36.05919115104277, 39.56043956043956, 35.88709634002859, 34.61538654227748, 39.33649289099526, 42.10526031292563, 34.50635208603153, 38.26086809093825, 40.13841015427547, 34.10565219337312, 37.741604600548044, 35.91418012687854, 36.30831621813538, 36.18307463010285,

In [26]:
for i,leaf in enumerate(leaf_ICW):
    firstvis_val[leaf] = []
    lastvis_val[leaf] = []
    for cluster in range(1, 11):
        firstvis, lastvis = first_last_array2var(tree, cluster, leaf, 'ICW', 'r')
        firstvis_val[leaf].append(firstvis)
        lastvis_val[leaf].append(lastvis)


In [27]:
for i,leaf in enumerate(circ):
    firstvis_val[leaf] = []
    lastvis_val[leaf] = []
    for cluster in range(1, 11):
        firstvis, lastvis = first_last_array(tree, cluster, leaf)
        firstvis_val[leaf].append(firstvis)
        lastvis_val[leaf].append(lastvis)


In [28]:
for i,leaf in enumerate(leaves):
    firstvis_val[leaf] = []
    lastvis_val[leaf] = []
    for cluster in range(1, 11):
        firstvis, lastvis = first_last_array(tree, cluster, leaf)
        firstvis_val[leaf].append(firstvis)
        lastvis_val[leaf].append(lastvis)

In [29]:
for i,leaf in enumerate(leaf_weight):
    firstvis_val[leaf] = []
    lastvis_val[leaf] = []
    for cluster in range(1, 11):
        firstvis, lastvis = first_last_array2var(tree, cluster, leaf, 'Weight', 'r')
        firstvis_val[leaf].append(firstvis)
        lastvis_val[leaf].append(lastvis)
    print(firstvis_val[leaf])

[[50.933786957767865, 52.6235361415257, 48.85411096014377, 48.79097263503126], [50.847457627118644, 44.82029831900886, 49.95102915004135, 44.25228753766771, 48.65168453602309, 47.14867685939305, 47.483589498180336, 53.22740824080704, 48.46885217004318, 49.05008440429254, 51.2422368342593, 51.72745019623359, 45.89632905002419, 50.28625880565326, 49.61154441726032, 47.634407781785534, 51.48304839888995, 53.33333249951973, 45.33493088233858, 47.87986062201134, 43.95721761938085, 49.08722340252585, 45.29540572727119, 45.057231971466535, 51.02739902004083, 48.728382542021045, 50.381681898000394, 51.658292990833075, 50.61452428721849, 49.42119188938995, 50.840339367019894, 51.15848005775996, 51.56118014693764, 51.83603890170336, 51.090343919336625, 46.48351564512148, 51.31048698765191, 52.90806973094932, 46.44549763033176, 42.81376308753865, 53.567935095109384, 48.40579562717014, 45.41522551295964, 53.75347468006216, 48.42319124757128, 51.21268944847589, 50.70993993283193, 51.727115671237144

[[49.066213042232135, 47.3764638584743, 51.14589330447043, 51.20902736496874], [49.152542372881356, 55.17970571344074, 50.04897084995865, 55.74770858166378, 51.34831546397691, 52.851327025227505, 52.516410501819664, 46.77259175919296, 51.53115185814836, 50.94991230149575, 48.757763165740705, 48.27255346470438, 54.1036709499758, 49.713737554368535, 50.388459816587996, 52.36559221821446, 48.516947560117245, 46.66666750048027, 54.66507368069651, 52.12014274786265, 56.04278238061915, 50.912780466335526, 54.70459427272881, 54.942768028533465, 48.97260533463649, 51.271617457978955, 49.618322261974946, 48.34170700916693, 49.38547571278151, 50.578804713729234, 49.15966464001518, 48.84151994224004, 48.43881985306237, 48.16396435593738, 48.90965905161155, 53.51648435487852, 48.68951685780915, 47.09193384756598, 53.55450236966824, 57.18623305143184, 46.432061175958886, 51.594204372829864, 54.584774487040356, 46.246525319937845, 51.57680487176021, 48.78731411001046, 49.29006006716808, 48.272884328

[[27.95699059369471, 27.050431812841282, 29.23420962625774, 29.124126268320182], [26.779661879027632, 30.76110026516582, 27.815866839340565, 30.72227856090531, 27.752809845999387, 29.633402531374138, 28.665208233635763, 26.017876421988113, 28.51108856398033, 28.497408575412877, 27.22567251810698, 26.58349479290147, 30.129589717355643, 27.576334710846684, 27.524971872428733, 29.13978535641906, 27.33050722460816, 25.792350143682764, 30.14354213264761, 29.328623372772096, 31.016042780748666, 28.397566362385877, 30.634572792728136, 30.176899542625335, 26.712328796949848, 28.280771489318308, 27.044711082224243, 26.83417162104468, 26.81564245810056, 27.871770602387897, 26.78571514436466, 26.87673734003108, 27.004219409282697, 26.64389510631343, 27.49221255066358, 30.109889690692608, 26.713710499231603, 25.984991706790773, 29.57346043880517, 32.48987599454328, 25.708698389534053, 28.695652911052612, 30.709342965906046, 25.393882512558275, 28.789418472509887, 26.772389533552186, 27.58620809723

[[31.975099590154272, 30.820172725136512, 33.37059966446015, 33.15421748488265], [31.864407641739493, 36.152221262768045, 32.71302842805103, 36.01220796223147, 32.921347457371404, 34.7250504417979, 33.916848449091866, 30.78450937385171, 33.57972572527186, 33.160622205437846, 32.09109781539471, 31.285987935598534, 35.4211660664629, 32.44274714687589, 32.630410784051975, 34.40860215053764, 32.309321511651355, 30.71038293056801, 35.64593275236162, 34.18728074757284, 36.47058660333807, 33.36714202334796, 35.995624700907285, 35.37981325687108, 31.849315187799377, 33.36724132123413, 32.06106935327859, 31.65829145728643, 31.95530768879299, 32.59127113046699, 31.82773131130864, 31.510657571070922, 31.476792605114383, 31.169940628194322, 31.93146569232243, 35.49450465611049, 31.552419556415295, 30.58161251478374, 34.59715639810427, 37.955464414667155, 30.40078147967889, 33.623187668657536, 35.6401395387239, 29.93512398543726, 33.875888304425715, 31.43656877080914, 32.55577988933354, 30.74265763

[[36.502547319733644, 35.20122129386912, 37.7864733261176, 37.82912370605719], [36.27118471652101, 40.803381709018275, 36.92458504051313, 40.895218224736965, 37.9775272326523, 38.900205651812136, 38.84026193363746, 34.657400778092196, 37.909189738939354, 37.478411383530705, 35.81780436921058, 35.60460610430657, 39.95680411413871, 36.736640151609464, 37.18091072956234, 38.60215217836441, 35.69915277353137, 34.53551954258987, 40.31100643305813, 38.33922499637738, 41.60427970682236, 37.829614416120336, 40.3719922427256, 40.58272697111683, 36.18721611314218, 37.741604600548044, 36.968378030604434, 35.577890980782826, 36.64804384029111, 37.3107750554269, 35.92437170090785, 36.05190081165019, 35.86497890295359, 35.69598615007521, 36.05919115104277, 39.56043956043956, 35.88709634002859, 34.61538654227748, 39.33649289099526, 42.10526031292563, 34.50635208603153, 38.26086809093825, 40.13841015427547, 34.10565219337312, 37.741604600548044, 35.91418012687854, 36.30831621813538, 36.18307463010285,

In [30]:
for i,leaf in enumerate(leaf_ICW):
    firstvis_val[leaf] = []
    lastvis_val[leaf] = []
    for cluster in range(1, 11):
        firstvis, lastvis = first_last_array2var(tree, cluster, leaf, 'ICW', 'r')
        firstvis_val[leaf].append(firstvis)
        lastvis_val[leaf].append(lastvis)


In [31]:
print(firstvis_val['BMI'][1])


[35.5, 36.0, 39.900001525878906, 38.400001525878906, 34.79999923706055, 37.0, 36.599998474121094, 43.599998474121094, 37.0, 37.79999923706055, 39.20000076293945, 42.29999923706055, 35.70000076293945, 41.5, 35.20000076293945, 37.70000076293945, 42.0, 41.79999923706055, 34.79999923706055, 38.29999923706055, 35.20000076293945, 39.5, 35.70000076293945, 38.5, 40.0, 34.400001525878906, 41.900001525878906, 38.400001525878906, 36.29999923706055, 38.900001525878906, 38.099998474121094, 40.599998474121094, 45.20000076293945, 42.0, 41.900001525878906, 35.5, 37.79999923706055, 41.099998474121094, 39.20000076293945, 38.099998474121094, 46.70000076293945, 42.0, 40.0, 42.099998474121094, 37.0, 38.400001525878906, 40.5, 45.79999923706055, 40.5, 40.0, 39.70000076293945, 40.20000076293945, 44.70000076293945, 41.400001525878906, 35.70000076293945, 38.900001525878906, 31.600000381469727, 38.099998474121094, 48.0, 35.70000076293945, 36.20000076293945, 36.900001525878906, 41.29999923706055, 37.2000007629394

In [ ]:
#print(len(firstvis_val['ICW'][0]))

In [32]:
header = 'Valore medio alla prima visita per il cluster: '
footer = 'Numero totale di pazienti '
for i in range(1, 11):
    header += ' & ' + str(i)
header += ' \\\\'
print(header)
for i,leaf in enumerate(circ):
    rows = var2lab(leaf) 
    for cluster in range(10):
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster])) 
    rows += '\\\\'
    print(rows)
for i,leaf in enumerate(leaves):
    rows = var2lab(leaf) 
    for cluster in range(10):
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster])) 
    rows += '\\\\'    
    print(rows)
for i,leaf in enumerate(leaf_weight):
    rows = var2lab(leaf) 
    for cluster in range(10):
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster])) 
    rows += '\\\\'    
    print(rows)
for i,leaf in enumerate(leaf_ICW):
    rows = var2lab(leaf) 
    for cluster in range(10):
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster])) 
    rows += '\\\\'    
    print(rows)
for i in range(10):
    footer += ' & ' + str(len(firstvis_val['Gender'][i]))
footer += ' \\\\'
print(footer)

Valore medio alla prima visita per il cluster:  & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 \\
Circonferenza braccio [cm] & 83.25  & 42.26  & 34.90  & 52.06  & 35.20  & 37.52  & 31.69  & 46.51  & 39.88  & 24.64 \\
Circonferenza collo [cm] & 50.30  & 44.47  & 37.96  & 50.59  & 38.93  & 40.27  & 35.03  & 46.77  & 42.69  & 29.24 \\
Circonferenza torace [cm] & 130.30  & 113.87  & 100.04  & 126.27  & 104.99  & 105.87  & 93.92  & 123.75  & 115.22  & 78.22 \\
Circonferenza addome [cm] & 153.53  & 119.84  & 100.17  & 134.27  & 99.17  & 107.73  & 88.40  & 137.94  & 120.62  & 66.56 \\
Circonferenza fianchi [cm] & 141.35  & 115.71  & 102.94  & 128.84  & 104.86  & 109.35  & 97.86  & 123.68  & 114.55  & 84.54 \\
Circonferenza coscia destra [cm] & 73.23  & 62.12  & 54.78  & 68.74  & 57.45  & 60.05  & 52.99  & 67.18  & 63.09  & 44.62 \\
Sesso & 0.00  & 0.89  & 0.99  & 0.62  & 0.21  & 0.90  & 0.94  & 0.04  & 0.08  & 0.65 \\
DeltaDays & 0.00  & 0.00  & 0.00  & 0.00  & 0.00  & 0.00  & 0.00  & 0.00  & 0.00 

In [24]:
leaves.remove('DeltaDays')

header = 'Mean values to the first visit: '
footer = 'Numero totale di pazienti '
for i in range(1):
    header += ' & ' + str(i)
header += ' \\\\'
print(header)
for i,leaf in enumerate(circ):
    rows = var2lab(leaf) 
    for cluster in range(1):
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster])) + ' & %.2f ' %(np.std(firstvis_val[leaf][cluster])) + ' & %.0f ' %(np.std(firstvis_val[leaf][cluster])/np.mean(firstvis_val[leaf][cluster])*100)
    rows += '\\\\'
    print(rows)
for i,leaf in enumerate(leaves):
    rows = var2lab(leaf) 
    for cluster in range(1):
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster])) + ' & %.2f ' %(np.std(firstvis_val[leaf][cluster])) + ' & %.0f ' %(np.std(firstvis_val[leaf][cluster])/np.mean(firstvis_val[leaf][cluster])*100) 
    rows += '\\\\'    
    print(rows)
for i,leaf in enumerate(leaf_weight):
    rows = var2lab(leaf) 
    for cluster in range(1):
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster])) + ' & %.2f ' %(np.std(firstvis_val[leaf][cluster])) + ' & %.0f ' %(np.std(firstvis_val[leaf][cluster])/np.mean(firstvis_val[leaf][cluster])*100) 
    rows += '\\\\'    
    print(rows)
for i,leaf in enumerate(leaf_ICW):
    rows = var2lab(leaf) 
    for cluster in range(1):
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster])) + ' & %.2f ' %(np.std(firstvis_val[leaf][cluster])) + ' & %.0f ' %(np.std(firstvis_val[leaf][cluster])/np.mean(firstvis_val[leaf][cluster])*100) 
    rows += '\\\\'    
    print(rows)
for i in range(1):
    footer += ' & ' + str(len(firstvis_val['Gender'][i]))
footer += ' \\\\'
print(footer)

Mean values to the first visit:  & 0 \\
Circonferenza braccio [cm] & 34.81  & 5.46  & 16 \\
Circonferenza collo [cm] & 37.88  & 4.24  & 11 \\
Circonferenza torace [cm] & 100.74  & 9.94  & 10 \\
Circonferenza addome [cm] & 98.25  & 15.56  & 16 \\
Circonferenza fianchi [cm] & 103.37  & 8.85  & 9 \\
Circonferenza coscia destra [cm] & 55.98  & 5.54  & 10 \\
Sesso & 0.74  & 0.44  & 59 \\
Altezza [cm] & 164.08  & 8.87  & 5 \\
Età & 38.38  & 14.98  & 39 \\
Peso [kg] & 79.76  & 18.59  & 23 \\
ICW [\%] & 22.62  & 4.77  & 21 \\
Proteine & 9.78  & 2.06  & 21 \\
Minerali & 3.45  & 0.72  & 21 \\
SLM & 46.87  & 9.78  & 21 \\
BMI [kg/m$^2$] & 29.49  & 5.75  & 20 \\
PBF & 36.71  & 8.87  & 24 \\
FFM braccio destro & 2.72  & 0.77  & 28 \\
FFM braccio sinistro & 2.69  & 0.76  & 28 \\
FFM tronco & 22.58  & 4.65  & 21 \\
FFM gamba destra & 7.51  & 1.72  & 23 \\
FFM gamba sinistra & 7.48  & 1.68  & 22 \\
BFM braccio destro & 2.52  & 1.73  & 69 \\
BFM braccio sinistro & 2.53  & 1.74  & 69 \\
BFM tronco & 15.

In [45]:
header = 'Valore medio all\'ultima visita per il cluster: '
footer = 'Numero totale di pazienti '
for i in range(1, 11):
    header += ' & ' + str(i)
header += ' \\\\'
print(header)
for i,leaf in enumerate(circ):
    rows = var2lab(leaf) 
    for cluster in range(10):
        rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][cluster])) 
    rows += '\\\\'
    print(rows)
for i,leaf in enumerate(leaves):
    rows = var2lab(leaf) 
    for cluster in range(10):
        rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][cluster])) 
    rows += '\\\\'    
    print(rows)
for i,leaf in enumerate(leaf_weight):
    rows = var2lab(leaf) 
    for cluster in range(10):
        rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][cluster])) 
    rows += '\\\\'    
    print(rows)
for i,leaf in enumerate(leaf_TBW):
    rows = var2lab(leaf) 
    for cluster in range(10):
        rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][cluster])) 
    rows += '\\\\'    
    print(rows)
for i in range(10):
    footer += ' & ' + str(len(lastvis_val['Gender'][i]))
footer += ' \\\\'
print(footer)

Valore medio all'ultima visita per il cluster:  & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 \\
Circonferenza braccio [cm] & 63.62  & 40.01  & 33.73  & 48.57  & 34.26  & 36.63  & 31.07  & 44.17  & 38.59  & 24.95 \\
Circonferenza collo [cm] & 47.98  & 42.63  & 36.91  & 47.83  & 38.33  & 39.39  & 34.58  & 44.19  & 41.71  & 29.48 \\
Circonferenza torace [cm] & 128.47  & 110.48  & 97.84  & 122.36  & 103.33  & 103.97  & 92.79  & 120.73  & 112.82  & 78.94 \\
Circonferenza addome [cm] & 147.38  & 114.71  & 96.07  & 132.13  & 95.63  & 104.47  & 86.20  & 132.41  & 114.68  & 66.92 \\
Circonferenza fianchi [cm] & 132.10  & 112.74  & 100.80  & 124.26  & 103.08  & 107.43  & 96.68  & 120.39  & 111.91  & 84.90 \\
Circonferenza coscia destra [cm] & 70.23  & 60.69  & 53.61  & 66.14  & 56.26  & 58.91  & 52.23  & 65.80  & 61.61  & 44.81 \\
Sesso & 0.00  & 0.89  & 0.99  & 0.62  & 0.21  & 0.90  & 0.94  & 0.04  & 0.08  & 0.65 \\
DeltaDays & 148.25  & 158.23  & 153.97  & 136.12  & 116.86  & 140.68  & 157.01  & 1

KeyError: 'ICW'

In [46]:
header = 'Valore medio alla prima visita per il cluster: '
footer = 'Numero totale di pazienti '

leaves = ['Gender', 'Age', 'Height', 'Weight', 'BMI', 'BFM', 'FFM']
cluster = 10

header += ' & ' + str(cluster) + ' \\\\'
print(header)
for i,leaf in enumerate(leaves):
    rows = var2lab(leaf) 
    rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster-1])) 
    rows += '\\\\'
    print(rows)
for i,leaf in enumerate(circ):
    rows = var2lab(leaf) 
    rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][cluster-1])) 
    rows += '\\\\'
    print(rows)
footer += ' & ' + str(len(firstvis_val['Gender'][cluster-1])) + ' \\\\'
print(footer)

Valore medio alla prima visita per il cluster:  & 10 \\
Sesso & 0.65 \\
Età & 22.90 \\
Altezza [cm] & 152.26 \\
Peso [kg] & 43.54 \\
BMI [kg/m$^2$] & 18.82 \\
BFM [\%] & 20.54 \\
FFM [\%] & 79.46 \\
Circonferenza braccio [cm] & 24.64 \\
Circonferenza collo [cm] & 29.24 \\
Circonferenza torace [cm] & 78.22 \\
Circonferenza addome [cm] & 66.56 \\
Circonferenza fianchi [cm] & 84.54 \\
Circonferenza coscia destra [cm] & 44.62 \\
Numero totale di pazienti  & 34 \\


In [47]:
header = 'Valore medio all\'ultima visita per il cluster: '
footer = 'Numero totale di pazienti '

leaves = ['Gender', 'Age', 'Height', 'Weight', 'BMI', 'BFM', 'FFM']
cluster = 5

header += ' & ' + str(cluster) + ' \\\\'
print(header)
for i,leaf in enumerate(leaves):
    rows = var2lab(leaf) 
    rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][cluster-1])) 
    rows += '\\\\'
    print(rows)
for i,leaf in enumerate(circ):
    rows = var2lab(leaf) 
    rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][cluster-1])) 
    rows += '\\\\'
    print(rows)
footer += ' & ' + str(len(lastvis_val['Gender'][cluster-1])) + ' \\\\'
print(footer)

Valore medio all'ultima visita per il cluster:  & 5 \\
Sesso & 0.21 \\
Età & 38.78 \\
Altezza [cm] & 173.85 \\
Peso [kg] & 83.29 \\
BMI [kg/m$^2$] & 27.60 \\
BFM [\%] & 26.56 \\
FFM [\%] & 73.44 \\
Circonferenza braccio [cm] & 34.26 \\
Circonferenza collo [cm] & 38.33 \\
Circonferenza torace [cm] & 103.33 \\
Circonferenza addome [cm] & 95.63 \\
Circonferenza fianchi [cm] & 103.08 \\
Circonferenza coscia destra [cm] & 56.26 \\
Numero totale di pazienti  & 300 \\


In [48]:
tab = '\\begin{tabular}{l' 
header = ' '
subheader = ' '
footer = 'Numero totale di pazienti '
circ = ['Chest_circ', 'Abdomen_circ', 'Hip_circ']

for leaf in circ:
    header += ' & \multicolumn{3}{|c}{' + var2lab(leaf) + '}'
    subheader += ' & iniziale & finale & $\\Delta$[\%] '
    tab += ' |c c c'
tab += '}'
header += ' \\\\'
subheader += ' \\\\'
print(tab)
print(header)
print(subheader)
for i in range(10):
    rows = 'Cluster ' + str(i+1)
    for leaf in circ:
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][i])) 
        rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][i])) 
        rows += ' & %.1f ' %((np.mean(lastvis_val[leaf][i])-np.mean(firstvis_val[leaf][i]))/np.mean(firstvis_val[leaf][i])*100)
    rows += '\\\\'
    print(rows)


\begin{tabular}{l |c c c |c c c |c c c}
  & \multicolumn{3}{|c}{Circonferenza torace [cm]} & \multicolumn{3}{|c}{Circonferenza addome [cm]} & \multicolumn{3}{|c}{Circonferenza fianchi [cm]} \\
  & iniziale & finale & $\Delta$[\%]  & iniziale & finale & $\Delta$[\%]  & iniziale & finale & $\Delta$[\%]  \\
Cluster 1 & 130.30  & 128.47  & -1.4  & 153.53  & 147.38  & -4.0  & 141.35  & 132.10  & -6.5 \\
Cluster 2 & 113.87  & 110.48  & -3.0  & 119.84  & 114.71  & -4.3  & 115.71  & 112.74  & -2.6 \\
Cluster 3 & 100.04  & 97.84  & -2.2  & 100.17  & 96.07  & -4.1  & 102.94  & 100.80  & -2.1 \\
Cluster 4 & 126.27  & 122.36  & -3.1  & 134.27  & 132.13  & -1.6  & 128.84  & 124.26  & -3.6 \\
Cluster 5 & 104.99  & 103.33  & -1.6  & 99.17  & 95.63  & -3.6  & 104.86  & 103.08  & -1.7 \\
Cluster 6 & 105.87  & 103.97  & -1.8  & 107.73  & 104.47  & -3.0  & 109.35  & 107.43  & -1.8 \\
Cluster 7 & 93.92  & 92.79  & -1.2  & 88.40  & 86.20  & -2.5  & 97.86  & 96.68  & -1.2 \\
Cluster 8 & 123.75  & 120.73  & 

In [58]:
cluster = [1, 8, 5, 9, 7, 3, 6, 2, 4, 10]

tab = '\\begin{tabular}{l' 
header = ' '
subheader = ' '
footer = 'Numero totale di pazienti '
nutri_idro = ["BMI", "WHR", "TBW", "ECW"]

for leaf in nutri_idro:
    header += ' & \multicolumn{3}{|c}{' + var2lab(leaf) + '}'
    subheader += ' & iniziale & finale & $\\Delta$[\%] '
    tab += ' |c c c'
tab += '}'
header += ' \\\\'
subheader += ' \\\\'
print(tab)
print(header)
print(subheader)
for i in cluster:
    rows = clust2tag(str(i) + "_")
    for leaf in nutri_idro:
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][i-1])) 
        rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][i-1])) 
        rows += ' & %.1f ' %((np.mean(lastvis_val[leaf][i-1])-np.mean(firstvis_val[leaf][i-1]))/np.mean(firstvis_val[leaf][i-1])*100)
    rows += '\\\\'
    print(rows)

\begin{tabular}{l |c c c |c c c |c c c |c c c}
  & \multicolumn{3}{|c}{BMI [kg/m$^2$]} & \multicolumn{3}{|c}{WHR} & \multicolumn{3}{|c}{TBW} & \multicolumn{3}{|c}{ECW [\%]} \\
  & iniziale & finale & $\Delta$[\%]  & iniziale & finale & $\Delta$[\%]  & iniziale & finale & $\Delta$[\%]  & iniziale & finale & $\Delta$[\%]  \\
1 - U, Obesità grave & 55.80  & 48.25  & -13.5  & 1.09  & 1.12  & 3.5  & 36.83  & 40.83  & 10.9  & 63.13  & 63.00  & -0.2 \\
8 - U, Molto Sovrappeso & 41.45  & 38.84  & -6.3  & 1.12  & 1.10  & -1.5  & 42.50  & 44.71  & 5.2  & 61.89  & 61.09  & -1.3 \\
5 - U, Leggero Sovrappeso & 28.72  & 27.60  & -3.9  & 0.94  & 0.93  & -1.9  & 52.29  & 53.81  & 2.9  & 60.27  & 60.35  & 0.1 \\
9 - U, Sovrappeso & 34.56  & 32.74  & -5.3  & 1.05  & 1.02  & -2.8  & 46.38  & 48.45  & 4.5  & 60.92  & 60.85  & -0.1 \\
7 - D, Normopeso, 35 anni & 26.04  & 25.29  & -2.9  & 0.90  & 0.89  & -1.3  & 48.06  & 49.41  & 2.8  & 61.56  & 61.48  & -0.1 \\
3 - D, Normopeso, 55 anni & 30.66  & 29.20  &

In [ ]:
cluster = [1, 8, 5, 9, 7, 3, 6, 2, 4, 10]

tab = '\\begin{tabular}{l' 
header = ' '
subheader = ' '
footer = 'Numero totale di pazienti '
adi_musc = ["", "WHR", "TBW", "ECW"]

for leaf in adi_musc:
    header += ' & \multicolumn{3}{|c}{' + var2lab(leaf) + '}'
    subheader += ' & iniziale & finale & $\\Delta$[\%] '
    tab += ' |c c c'
tab += '}'
header += ' \\\\'
subheader += ' \\\\'
print(tab)
print(header)
print(subheader)
for i in cluster:
    rows = clust2tag(str(i) + "_")
    for leaf in adi_musc:
        rows += ' & %.2f ' %(np.mean(firstvis_val[leaf][i-1])) 
        rows += ' & %.2f ' %(np.mean(lastvis_val[leaf][i-1])) 
        rows += ' & %.1f ' %((np.mean(lastvis_val[leaf][i-1])-np.mean(firstvis_val[leaf][i-1]))/np.mean(firstvis_val[leaf][i-1])*100)
    rows += '\\\\'
    print(rows)